# Neural Network from Scratch Compared to Tensorflow

Contains functions and classes to create a feed-forward neural network from scratch including activation functions, loss functions, regularization, and Layer, Model and Metric classes.

Trains two models with the same architecture: one built "from-scratch" using the classes and functions here, and one built with TensorFlow.

## Setup
Imports, read in data

In [ ]:
import numpy as np  # linear algebra
import pandas as pd  # data processing
import matplotlib.pyplot as plt  # plotting

import math
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import warnings

import tensorflow as tf  # model

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
DATASET_FILENAME = "/content/drive/My Drive/ML6140  - Project/Data/enhanced_data_df1.csv"

In [ ]:
data = pd.read_csv(DATASET_FILENAME)
data.head()

,incident_name,incident_created_year,incident_created_month,incident_created_day,incident_created_hour,incident_created_minute,incident_latitude,incident_longitude,incident_geohash,LCD_station_id,...,60_meanRelativeHumidity,60_minRelativeHumidity,60_maxRelativeHumidity,60_maxWindSpeed,60_calculate_circular_meanWindDirection,60_mode_functionWindDirection,60_mode_functionSkyConditions,far_hist_avg_acres_burned,near_hist_avg_acres_burned,class_label
0,Stephens Fire,2015,2,24,12,15,41.485000,-121.851000,9r3k50n,24215,...,68.275119,45.700000,84.450000,8.000000,12.517703,0.0,CLR:00,39.6260,136.333333,1
1,Ward Fire,2015,4,13,5,30,40.050833,-120.701667,9r45v75,23225,...,44.454664,27.850000,62.950000,11.666667,73.118840,0.0,CLR:00,1444.3735,46.090000,1
2,Highway Fire,2015,4,18,18,12,33.884313,-117.642759,9qh2fbr,53175,...,63.415999,28.833333,92.200000,13.783333,348.550492,0.0,CLR:00,3083.3440,370.770000,1
3,Bald Fire,2015,6,10,7,42,41.151300,-123.829200,9prfn5q,24283,...,90.247624,74.383333,99.483333,14.566667,326.357672,0.0,CLR:00,166.6760,17.000000,1
4,Saddle Fire,2015,6,10,15,0,40.924000,-123.168000,9r22v1z,117,...,56.194739,22.550000,89.833333,10.733333,342.513313,0.0,CLR:00,1696.2415,515.600000,1


In [ ]:
data = data.drop(["incident_name", "GHCN_station_id", "LCD_station_id"], axis=1)

In [ ]:
data = data.drop(["mode_functionSkyConditions", "10_mode_functionSkyConditions",
                  "30_mode_functionSkyConditions",
                  "60_mode_functionSkyConditions", "incident_geohash"],
                 axis=1)

In [ ]:
train, validate = train_test_split(data)

In [ ]:
# Get as np arrays instead of dataframes
train_x = train.drop(["class_label"], axis=1).values
train_y = train["class_label"].values

validate_x = validate.drop(["class_label"], axis=1).values
validate_y = validate["class_label"].values

## Model

### From Scratch

#### Classes and functions to make the neural network

In [ ]:
np.seterr(all="warn")

{'divide': 'warn', 'over': 'warn', 'under': 'warn', 'invalid': 'warn'}

In [ ]:
FLOAT_TYPE = np.float64
# The sigmoid function isn't precise enought to avoid giving 1 or 0,
# so correct with near-zero
SMALLEST = np.finfo(FLOAT_TYPE).smallest_normal
LARGEST = np.finfo(FLOAT_TYPE).max

NEAR_0 = np.nextafter(np.float32(0), np.float32(1)).astype(np.float64)
NEAR_1 = np.nextafter(np.float32(1), np.float32(0)).astype(np.float64)

<ipython-input-42-cc5f2a7329af>:7: RuntimeWarning: underflow encountered in nextafter
  NEAR_0 = np.nextafter(np.float32(0), np.float32(1)).astype(np.float64)


Weight initialization functions

In [ ]:
# https://www.deeplearning.ai/ai-notes/initialization/index.html
# https://wandb.ai/sauravmaheshkar/initialization/reports/A-Gentle-Introduction-To-Weight-Initialization-for-Neural-Networks--Vmlldzo2ODExMTg

def he_init(shape):
  std = 2 / shape[0]  # TODO: assumes this is num rows
  return np.random.default_rng().normal(0, std, shape)

def xavier_init(shape):
  # shape of weights matrix (in x out) gives in and out dims
  std = 2 / sum(shape)
  return np.random.default_rng().normal(0, std , shape)

Activation functions

In [ ]:
# Functions (for forward prop)
def scalar_relu(x):
  return max(0, x)

def scalar_sigmoid(x):
  try:
    ex = math.exp(-x)
    val = 1 / (1 + ex)
    val = max(val, NEAR_0)
    val = min(val, NEAR_1)
    return val
  except (FloatingPointError, ValueError) as err:  # Underflow if x is large
    return NEAR_1
  except OverflowError as err:  # Overflow error if x is small
    # 1 / (1 + max) will likely give an underflow error.
    return NEAR_0

def scalar_identity(x):
  return x

alpha = 0.0001

def scalar_leaky_relu(x):
  if x > 0:
    return x
  try:
    val = alpha * x
    if np.isnan(val):
      return -SMALLEST
    return val
  except FloatingPointError as err:
    return -SMALLEST

relu = np.vectorize(scalar_relu)
sigmoid = np.vectorize(scalar_sigmoid)
identity = np.vectorize(scalar_identity)
leaky_relu = np.vectorize(scalar_leaky_relu)

# Derivatives (for backprop)
def scalar_relu_grad(z):
  if z > 0:
    return 1
  return 0

def scalar_sigmoid_grad(z):
  return scalar_sigmoid(z) * (1 - scalar_sigmoid(z))

def scalar_identity_grad(y):
  return 1

def scalar_leaky_relu_grad(z):
  if z > 0:
    return 1
  return alpha

relu_grad = np.vectorize(scalar_relu_grad)
sigmoid_grad = np.vectorize(scalar_sigmoid_grad)
identity_grad = np.vectorize(scalar_identity_grad)
leaky_relu_grad = np.vectorize(scalar_leaky_relu_grad)

activations_dict = {
    "relu": {
      "function": relu,
      "gradient": relu_grad,
      "weight_init_method": he_init
    },
    "sigmoid": {
      "function": sigmoid,
      "gradient": sigmoid_grad,
      "weight_init_method": xavier_init
    },
    "identity": {
      "function": identity,
      "gradient": identity_grad,
      "weight_init_method": he_init
    },
    "leaky_relu": {
      "function": leaky_relu,
      "gradient": leaky_relu_grad,
      "weight_init_method": he_init
    }
}

Loss functions

In [ ]:
ln = np.vectorize(math.log)

def scalar_binary_crossentropy(y, pred_y):
  try:
    loss = (y * ln(pred_y) + (1 - y) * ln(1 - pred_y))
  except ValueError as err:
    msg = f"WARNING: Caught error when trying to take the log of {pred_y} and {1 - pred_y}."
    if pred_y == 1:
      loss = (y * ln(pred_y) + (1 - y) * ln(SMALLEST))
    else:
      loss = (y * ln(SMALLEST) + (1 - y) * ln(1 - pred_y))
    msg += f" Taking log of {SMALLEST} instead of 0."
    warnings.warn(msg)
  return -loss

# TODO: agg version?
def scalar_binary_crossentropy_grad(y, pred_y):
  try:
    return ((1 - y) / (1 - pred_y)) - (y / pred_y)
  except (FloatingPointError, ZeroDivisionError) as err:
    msg = f"WARNING: Caught error dividing by 1 - {pred_y} or by {pred_y}."
    msg += f" Dividing by {SMALLEST} instead."
    warnings.warn(msg)
    return ((1 - y) / (max(1 - pred_y, SMALLEST))) - (y / max(pred_y, SMALLEST))

loss_func_dict = {
    "binary_crossentropy": {
        "function": scalar_binary_crossentropy,
        "gradient": scalar_binary_crossentropy_grad
    }
}

Regularization functions

In [ ]:
def regularization_none(weights, lmb, num_samples):
  return 0

def regularization_l2(weights, lmb, num_samples):
  return weights * (2 * lmb / num_samples)

regularization_gradients = {
    "none": regularization_none,
    "l2": regularization_l2,
}

Metrics

In [ ]:
class Metric:
  def __init__(self, y_array, pred_y_array):
    self.y_array = y_array
    self.pred_y_array = pred_y_array

    df = pd.concat([pd.DataFrame(y_array, columns=["y"]),
                  pd.DataFrame(pred_y_array, columns=["pred_y"])],
                 axis=1)

    self.tp = len(df[(df.y == 1) & (df.pred_y == 1)])
    self.tn = len(df[(df.y == 0) & (df.pred_y == 0)])
    self.fp = len(df[(df.y == 0) & (df.pred_y == 1)])
    self.fn = len(df[(df.y == 1) & (df.pred_y == 0)])
    self.total = len(df)

  def true_positive_rate(self):
    try:
      return self.tp / self.total
    except ZeroDivisionError:
      return np.nan


  def true_negative_rate(self):
    try:
      return self.tn / self.total
    except ZeroDivisionError:
      return np.nan


  def false_positive_rate(self):
    try:
      return self.fp / self.total
    except ZeroDivisionError:
      return np.nan


  def false_negative_rate(self):
    try:
      return self.fn / self.total
    except ZeroDivisionError:
      return np.nan


  def confusion_matrix(self, rates=False):
    if rates:
      return self._confusion_matrix(self.true_positive_rate(),
                                    self.true_negative_rate(),
                                    self.false_positive_rate(),
                                    self.false_negative_rate())
    return self._confusion_matrix(self.tp, self.tn, self.fp, self.fn)


  def _confusion_matrix(self, tp, tn, fp, fn):
    mat = pd.DataFrame([[tp, fn], [fp, tn]],
                       columns=["Predicted Positive", "Predicted Negative"],
                       index=["Positive", "Negative"])
    return mat


  def precision(self):
    try:
      den = self.tp + self.fp
      return self.tp / den
    except ZeroDivisionError:
      return np.nan


  def recall(self):
    try:
      den = self.tp + self.fn
      return self.tp / den
    except ZeroDivisionError:
      return np.nan


  def accuracy(self):
    try:
      trues = self.tp + self.tn
      return trues / self.total
    except ZeroDivisionError:
      return np.nan


  def f1_score(self):
    try:
      den = 2 * self.tp + self.fp + self.fn
      return 2 * self.tp / den
    except ZeroDivisionError:
      return np.nan

Model

In [ ]:
class Layer:
  """
  Dense layer for fully connected neural network.
  """
  def __init__(self, input_size, output_size, activation="identity", name=""):
    self.activation_name = activation
    self.weights = np.atleast_2d(activations_dict[activation]["weight_init_method"]((input_size, output_size))).astype(np.float128)
    self.activation_function = activations_dict[activation]["function"]
    self.activation_gradient = activations_dict[activation]["gradient"]

    self.bias = np.atleast_2d(np.zeros((1, output_size))).astype(np.float128)

    self.name=name

    # TODO: could instead push to a stack for training forward and backprop?
    self.x = np.atleast_2d(np.zeros((1, input_size))).astype(np.float128)  # input to layer
    self.z = np.atleast_2d(np.zeros((1, output_size))).astype(np.float128)  # intermediate value

    # don't need this
    self.a = np.atleast_2d(np.zeros((1, output_size))).astype(np.float128)  # output / after activation


  def forward(self, x):  # np array or list. horizontal.
    # x = x-vector or output a of previous layer = input

    self.x = np.atleast_2d(np.array(x)).astype(np.float128)
    self.z = (np.matmul(self.x, self.weights) + self.bias).astype(np.float128)
    if np.isnan(self.z[0][0]):
      raise Exception("NaN z")  # Without proper regularization, could explode
    self.a = self.activation_function(self.z).astype(np.float128)
    return self.a


  # Regularization is key, otherwise exploding values.
  def backprop(self, upstream_gradient_vector, lr=0.001, regularization="l2", reg_lambda=0.015, num_samples=1):
    # update local weights and bias
    gradient_z = np.multiply(upstream_gradient_vector, self.activation_gradient(self.z)).astype(np.float128)
    if np.isnan(gradient_z[0][0]):
      raise Exception("NaN gradient_z")
    temp_weights = (self.weights - lr * (np.matmul(self.x.T, gradient_z))).astype(np.float128)
    temp_weights = temp_weights - lr * regularization_gradients[regularization](temp_weights, reg_lambda, num_samples)
    if np.isnan(temp_weights[0][0]):
      raise Exception("NaN weights")
    self.weights = temp_weights
    self.bias = (self.bias - lr * gradient_z).astype(np.float128)
    gradient_x = np.matmul(gradient_z, self.weights.T)
    return gradient_x


  def input_size(self):
    return self.weights.shape[0]


  def output_size(self):
    return self.weights.shape[1]


  def __str__(self):
    return f"Layer {self.name}. Input size {self.input_size()}, output size {self.output_size()}. Activation {self.activation_name}."


  def summary(self):
    df = pd.DataFrame([[self.name, self.input_size(), self.output_size(), self.activation_name]],
                      columns=["layer name", "input_size", "output_size", "activation"])
    return df

  def display_summary(self):
    display(self.summar())

In [ ]:
class Model:
  def __init__(self, loss):
    self.set_loss_function(loss)
    self.layers = []
    self.set_final_prediction_function()


  def set_loss_function(self, loss_name):
    try:
      self.loss_function = loss_func_dict[loss_name]["function"]
      self.loss_gradient = loss_func_dict[loss_name]["gradient"]
      self.loss_name = loss_name
    except KeyError as err:
      raise(f"Cannot find known loss function named {loss_name}") from err


  def set_final_prediction_function(self):
    self.final_prediction_function = lambda x: int(x > .5)
    self.final_prediction_function_name = "threshold at .5"


  def add_layer(self, layer):
    if isinstance(layer, Layer):
      # TODO: more validation of layer dimensions
      self.layers.append(layer)


  def num_layers(self):
    return len(self.layers)


  def forwardprop_single(self, x):
    a = np.atleast_2d(x)  # In case not already np array
    for layer in self.layers:
      try:
        a = layer.forward(a)
      except Exception as err:
        print("layer: ", layer.name)
        raise err
    return a


  def backprop_single(self, y, pred_y, regularization="l2", reg_lambda=0.015, num_samples=1):
    grad = np.atleast_2d(self.loss_gradient(y, pred_y))
    for i in range(len(self.layers) - 1, -1, -1):
      grad = self.layers[i].backprop(grad, reg_lambda=reg_lambda)


  def train(self, x_array, y_array, epochs=1, lr=0.001, regularization="l2", reg_lambda=0.015, num_samples=1):
    """
    x: list or array of lists or arrays
    y: list or array of 1s and 0s
    """
    # TODO: with stack instead of z and a stored in layers?
    if len(x_array) != len(y_array):
      raise ValueError("x and y arrays must be of the same length.")
    if lr <= 0:
      raise ValueError("Learning rate must be positive.")

    for i in range(max(epochs, 1)):
      epoch_total_loss = 0
      for j in tqdm(range(len(y_array)), desc=f"Epoch {i + 1}/{epochs}"):
        x = x_array[j]
        y = y_array[j]
        y_pred = self.forwardprop_single(x)[0][0]
        self.backprop_single(y, y_pred, reg_lambda=reg_lambda)
        epoch_total_loss += self.loss_function(y, y_pred)
      print(f" -- Loss: {epoch_total_loss / len(y_array)}")


  def predict_single(self, x):
    return self.final_prediction_function(self.forwardprop_single(x))

  def predict(self, x_df):
    x_df = pd.DataFrame(x_df)
    return np.array(x_df.apply(lambda x: self.predict_single(np.array(x)), axis=1))


  def eval(self, x_arr, y_arr):
    pred_y = self.predict(x_arr)
    metric = Metric(y_arr, pred_y)
    print(f"Precision: {metric.precision()}")
    print(f"Recall: {metric.recall()}")
    print(f"Accuracy: {metric.accuracy()}")
    print(f"F1 Score: {metric.f1_score()}")
    display(metric.confusion_matrix())
    return metric


  def __str__(self):
    outstring = f"Model with {self.num_layers()} layers."
    if len(self.layers):
      outstring += f"\nInput size {self.layers[0].input_size()}"
      outstring += f"\nOutput size {self.layers[-1].output_size()}"
    return outstring


  def summary(self):
    outstring = self.__str__()
    if len(self.layers):
      outstring += f"\nOutput activation {self.layers[-1].activation_name}"
      if not self.final_prediction_function_name == "identity":
        outstring += f" with {self.final_prediction_function_name}"
      outstring += f"\nLoss function {self.loss_name}"
      layers_df = pd.concat([layer.summary() for layer in self.layers],
                            ignore_index=True)
      outstring += f"\n\n{layers_df}"
    return outstring

  def display_summary(self):
    print(self.summary())

#### Building and training a model

In [ ]:
my_model = Model("binary_crossentropy")
my_model.add_layer(Layer(len(train_x[0]), 256, activation="leaky_relu", name="layer 1"))
my_model.add_layer(Layer(256, 512, activation="leaky_relu", name="layer 2"))
my_model.add_layer(Layer(512, 512, activation="leaky_relu", name="layer 3"))
my_model.add_layer(Layer(512, 256, activation="leaky_relu", name="layer 4"))
my_model.add_layer(Layer(256, 128, activation="leaky_relu", name="layer 5"))
my_model.add_layer(Layer(128, 64, activation="leaky_relu", name="layer 6"))
my_model.add_layer(Layer(64, 32, activation="leaky_relu", name="layer 7"))
my_model.add_layer(Layer(32, 1, activation="sigmoid", name="layer 8"))

In [ ]:
print(my_model.summary())

Model with 8 layers.
Input size 79
Output size 1
Output activation sigmoid with threshold at .5
Loss function binary_crossentropy

  layer name  input_size  output_size  activation
0    layer 1          79          256  leaky_relu
1    layer 2         256          512  leaky_relu
2    layer 3         512          512  leaky_relu
3    layer 4         512          256  leaky_relu
4    layer 5         256          128  leaky_relu
5    layer 6         128           64  leaky_relu
6    layer 7          64           32  leaky_relu
7    layer 8          32            1     sigmoid


In [ ]:
my_model.train(train_x, train_y, epochs=30, reg_lambda=0.01)

Epoch 1/30: 100%|██████████| 3664/3664 [02:09<00:00, 28.25it/s]


 -- Loss: 0.6581224664995367


Epoch 2/30: 100%|██████████| 3664/3664 [02:12<00:00, 27.68it/s]


 -- Loss: 0.633034586221378


Epoch 3/30: 100%|██████████| 3664/3664 [02:09<00:00, 28.27it/s]


...

 -- Loss: 0.6276939746870676


Epoch 29/30: 100%|██████████| 3664/3664 [02:08<00:00, 28.52it/s]


 -- Loss: 0.6276939508125334


Epoch 30/30: 100%|██████████| 3664/3664 [02:10<00:00, 28.02it/s]

 -- Loss: 0.6276939298412106


In [ ]:
my_model.eval(validate_x, validate_y)

0 809 0 413 1222
Precision: nan
Recall: 0.0
Accuracy: 0.6620294599018003
F1 Score: 0.0


,Predicted Positive,Predicted Negative
Positive,0,413
Negative,0,809


In [ ]:
train.to_csv("/content/drive/My Drive/ML6140  - Project/Model/Mea/train.csv", index=False)
validate.to_csv("/content/drive/My Drive/ML6140  - Project/Model/Mea/validate.csv", index=False)

### With Tensorflow

#### Match model from scratch

In [ ]:
tf_model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(512, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(512, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(256, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(128, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(64, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(31, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(1, activation=tf.keras.activations.sigmoid, use_bias=True)
])

In [ ]:
tf_model.compile(loss=tf.keras.metrics.binary_crossentropy, optimizer=tf.keras.optimizers.SGD(learning_rate=0.001))

In [ ]:
history = tf_model.fit(train_x, train_y, epochs=30)

Epoch 1/30
115/115 [==============================] - 3s 10ms/step - loss: 2.3031
Epoch 2/30
115/115 [==============================] - 1s 10ms/step - loss: 1.6013
Epoch 3/30
115/115 [==============================] - 2s 14ms/step - loss: 1.0118
Epoch 4/30
115/115 [==============================] - 2s 14ms/step - loss: 1.2840
Epoch 5/30
115/115 [==============================] - 2s 15ms/step - loss: 0.7099
Epoch 6/30
115/115 [==============================] - 2s 14ms/step - loss: 0.5862
Epoch 7/30
115/115 [==============================] - 2s 14ms/step - loss: 0.5790
Epoch 8/30
115/115 [==============================] - 1s 10ms/step - loss: 0.5791
Epoch 9/30
115/115 [==============================] - 1s 10ms/step - loss: 0.5468
Epoch 10/30
115/115 [==============================] - 1s 10ms/step - loss: 0.5701
Epoch 11/30
115/115 [==============================] - 1s 10ms/step - loss: 0.5139
Epoch 12/30
115/115 [==============================] - 1s 10ms/step - loss: 0.5011
Epoch 13/30
1

In [ ]:
tf_model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_30 (Dense)            (None, 256)               20480     
                                                                 
 dense_31 (Dense)            (None, 512)               131584    
                                                                 
 dense_32 (Dense)            (None, 512)               262656    
                                                                 
 dense_33 (Dense)            (None, 256)               131328    
                                                                 
 dense_34 (Dense)            (None, 128)               32896     
                                                                 
 dense_35 (Dense)            (None, 64)                8256      
                                                                 
 dense_36 (Dense)            (None, 31)               

In [ ]:
tf_pred_y = tf_model.predict(validate_x)
print(tf_pred_y)

39/39 [==============================] - 1s 8ms/step
[[0.2658078 ]
 [0.26055294]
 [0.06335828]
 ...
 [0.02768142]
 [0.14039499]
 [0.01729643]]


In [ ]:
tf_pred_y = np.where(tf_pred_y > .5, 1, 0)
tf_pred_y

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]])

In [ ]:
tf_metric = Metric(validate_y, tf_pred_y)
print(f"Precision: {tf_metric.precision()}")
print(f"Recall: {tf_metric.recall()}")
print(f"Accuracy: {tf_metric.accuracy()}")
print(f"F1 Score: {tf_metric.f1_score()}")
display(tf_metric.confusion_matrix())

Precision: 0.7582417582417582
Recall: 0.5012106537530266
Accuracy: 0.7774140752864157
F1 Score: 0.6034985422740525


,Predicted Positive,Predicted Negative
Positive,207,206
Negative,66,743
